In [1]:
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer

In [2]:
afq_dataset = AFQDataset.from_files(
    fn_nodes="../data/raw/combined_tract_profiles.csv",
    fn_subjects="../data/raw/participants_updated_id.csv",
    dwi_metrics=["dki_fa", "dki_md"],
    target_cols=["age"],
    index_col="subject_id",
)

In [3]:
afq_dataset.drop_target_na()

In [4]:
print(len(afq_dataset.subjects))
print(afq_dataset.X.shape)
print(afq_dataset.y.shape)

1867
(1867, 4800)
(1867,)


In [5]:
full_dataset = list(afq_dataset.as_tensorflow_dataset().as_numpy_iterator())

2022-02-09 15:46:57.861563: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
X = np.concatenate([xx[0][None] for xx in full_dataset], 0)

In [7]:
y = np.array([yy[1] for yy in full_dataset])

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# Split the data into train and test sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
X_train.shape, X_test.shape

((1493, 100, 48), (374, 100, 48))

In [11]:
imputer = SimpleImputer(strategy="median")

In [12]:
# Impute train and test separately:
X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)

In [13]:
X_train.shape, X_test.shape

((1493, 100, 48), (374, 100, 48))

In [ ]:
model_cnn_lenet = cnn_lenet(input_shape=(100, 48), n_classes=1, output_activation=None, verbose=True)
model_cnn_lenet.compile(loss= "mean_squared_error", optimizer="adam", metrics=["mean_squared_error"])
history_lenet = model_cnn_lenet.fit(X_train, y_train, epochs=2000, batch_size=128, validation_split=0.2)

Epoch 1/1000
10/10 [==============================] - 1s 49ms/step - loss: 100.7533 - mean_squared_error: 100.7533 - val_loss: 41.7785 - val_mean_squared_error: 41.7785
Epoch 2/1000
10/10 [==============================] - 0s 24ms/step - loss: 31.7558 - mean_squared_error: 31.7558 - val_loss: 12.0601 - val_mean_squared_error: 12.0601
Epoch 3/1000
10/10 [==============================] - 0s 23ms/step - loss: 22.3377 - mean_squared_error: 22.3377 - val_loss: 14.2346 - val_mean_squared_error: 14.2346
Epoch 4/1000
10/10 [==============================] - 0s 23ms/step - loss: 21.2473 - mean_squared_error: 21.2473 - val_loss: 12.3267 - val_mean_squared_error: 12.3267
Epoch 5/1000
10/10 [==============================] - 0s 26ms/step - loss: 19.3127 - mean_squared_error: 19.3127 - val_loss: 12.7528 - val_mean_squared_error: 12.7528
Epoch 6/1000
10/10 [==============================] - 0s 25ms/step - loss: 20.1536 - mean_squared_error: 20.1536 - val_loss: 12.8016 - val_mean_squared_error: 12.8

Epoch 50/1000
10/10 [==============================] - 0s 26ms/step - loss: 13.1653 - mean_squared_error: 13.1653 - val_loss: 8.9582 - val_mean_squared_error: 8.9582
Epoch 51/1000
10/10 [==============================] - 0s 26ms/step - loss: 12.6300 - mean_squared_error: 12.6300 - val_loss: 9.0786 - val_mean_squared_error: 9.0786
Epoch 52/1000
10/10 [==============================] - 0s 27ms/step - loss: 12.7993 - mean_squared_error: 12.7993 - val_loss: 9.8290 - val_mean_squared_error: 9.8290
Epoch 53/1000
10/10 [==============================] - 0s 25ms/step - loss: 13.4458 - mean_squared_error: 13.4458 - val_loss: 8.8688 - val_mean_squared_error: 8.8688
Epoch 54/1000
10/10 [==============================] - 0s 26ms/step - loss: 13.6119 - mean_squared_error: 13.6119 - val_loss: 8.9713 - val_mean_squared_error: 8.9713
Epoch 55/1000
10/10 [==============================] - 0s 24ms/step - loss: 12.7313 - mean_squared_error: 12.7313 - val_loss: 8.8228 - val_mean_squared_error: 8.8228
Epoc

10/10 [==============================] - 0s 23ms/step - loss: 10.3043 - mean_squared_error: 10.3043 - val_loss: 9.0188 - val_mean_squared_error: 9.0188
Epoch 149/1000
10/10 [==============================] - 0s 24ms/step - loss: 9.8401 - mean_squared_error: 9.8401 - val_loss: 8.3819 - val_mean_squared_error: 8.3819
Epoch 150/1000
10/10 [==============================] - 0s 22ms/step - loss: 9.9795 - mean_squared_error: 9.9795 - val_loss: 8.0577 - val_mean_squared_error: 8.0577
Epoch 151/1000
10/10 [==============================] - 0s 21ms/step - loss: 10.3118 - mean_squared_error: 10.3118 - val_loss: 7.5792 - val_mean_squared_error: 7.5792
Epoch 152/1000
10/10 [==============================] - 0s 23ms/step - loss: 10.8130 - mean_squared_error: 10.8130 - val_loss: 7.6348 - val_mean_squared_error: 7.6348
Epoch 153/1000
10/10 [==============================] - 0s 24ms/step - loss: 9.8702 - mean_squared_error: 9.8702 - val_loss: 7.7817 - val_mean_squared_error: 7.7817
Epoch 154/1000
10/1

Epoch 247/1000
10/10 [==============================] - 0s 23ms/step - loss: 7.8485 - mean_squared_error: 7.8485 - val_loss: 7.2891 - val_mean_squared_error: 7.2891
Epoch 248/1000
10/10 [==============================] - 0s 23ms/step - loss: 7.8043 - mean_squared_error: 7.8043 - val_loss: 7.1937 - val_mean_squared_error: 7.1937
Epoch 249/1000
10/10 [==============================] - 0s 22ms/step - loss: 7.7707 - mean_squared_error: 7.7707 - val_loss: 7.2338 - val_mean_squared_error: 7.2338
Epoch 250/1000
10/10 [==============================] - 0s 26ms/step - loss: 7.5982 - mean_squared_error: 7.5982 - val_loss: 7.1450 - val_mean_squared_error: 7.1450
Epoch 251/1000
10/10 [==============================] - 0s 26ms/step - loss: 8.0868 - mean_squared_error: 8.0868 - val_loss: 7.3113 - val_mean_squared_error: 7.3113
Epoch 252/1000
10/10 [==============================] - 0s 26ms/step - loss: 8.0559 - mean_squared_error: 8.0559 - val_loss: 7.2220 - val_mean_squared_error: 7.2220
Epoch 253/

In [ ]:
eval_cnn_lenet = model_cnn_lenet.evaluate(X_test, y_test)